In [1]:
# July 2023
# Implementing VoxNet for Autoencoder (not classification that explained previously):
# you can see the VoxNet's architecture at https://github.com/AutoDeep/VoxNet

In [2]:
# VoxnetEncoder is a name I myself selected here. I'm not sure if there is something similar out
# there for this or not...

# 1) Voxeling Modelnet10

In [3]:
dataset_addr = '/home/sbn/Downloads/datasets/data_/ModelNet10/'

In [4]:
import torch
from torch.utils.data import DataLoader
import numpy as np




# import sys
# sys.path.insert(0, dataset_addr)
from modelnet10 import ModelNet10

CLASSES = {
    0: 'bathtub',
    1: 'chair',
    2: 'dresser',
    3: 'night_stand',
    4: 'sofa',
    5: 'toilet',
    6: 'bed',
    7: 'desk',
    8: 'monitor',
    9: 'table'
}
N_CLASSES = len(CLASSES)



train_dataset = ModelNet10(data_root= dataset_addr, 
                           n_classes=N_CLASSES, 
                           idx2cls=CLASSES, 
                           split='train')

train_dataloader = DataLoader(train_dataset, 
                              batch_size=1, 
                              shuffle=True)




{'bathtub': 0, 'chair': 1, 'dresser': 2, 'night_stand': 3, 'sofa': 4, 'toilet': 5, 'bed': 6, 'desk': 7, 'monitor': 8, 'table': 9}


# Testing voxel

In [5]:
for sample in train_dataloader:
    voxel, cls_idx = sample['voxel'], sample['cls_idx']
    print(voxel.shape)
    break

torch.Size([1, 1, 32, 32, 32])


# VoxNet Encoder/ Decoder

<!-- ![image](images/VoxNet.png) -->

In [6]:
import torch
from torch import nn

In [7]:
# VoxNetEncoder = nn.Sequential(
#                 nn.Conv3d(in_channels=1, out_channels=32, kernel_size=(5,5,5), stride=2),
#                 nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), stride=1),
#                 nn.AvgPool3d(kernel_size=(2,2,2), stride=2),
#                 nn.Flatten(start_dim=1),
#                 nn.Linear(6912, 128),
#                 nn.Linear(128, 25),
#                 nn.Linear(25,3),
#                 nn.ReLU()
)
    

SyntaxError: unmatched ')' (3046689517.py, line 10)

In [8]:
# VoxNetDecoder = nn.Sequential(
#                 nn.Linear(3,25),
#                 nn.Linear(25, 128),
#                 nn.Linear(128, 1000),
#                 nn.Unflatten(dim=1, unflattened_size=(10,10,10)),

#                 nn.ConvTranspose3d(in_channels=1, out_channels=32, kernel_size=(7,7,7), dilation=2),
#                 nn.ReLU(),
#                 nn.ConvTranspose3d(in_channels=32, out_channels=32, kernel_size=(5,5,5), dilation=2),
#                 nn.ReLU(),
#                 nn.ConvTranspose3d(in_channels=32, out_channels=1, kernel_size=(5,5,5), dilation=1, 
#                                    output_padding=0,
#                                   padding=1,
#                                    stride=1),     
#                 nn.Tanh()
# )
    

In [9]:
# VoxNetAutoencoder = nn.Sequential(
#                 VoxNetEncoder(),
#                 VoxNetDecoder(),
# )

In [10]:
class VoxNetAutoencoder(nn.Module):
    def __init__(self):
        super(VoxNetAutoencoder, self).__init__()
        
        self.VoxNetEncoder = nn.Sequential(
                nn.Conv3d(in_channels=1, out_channels=32, kernel_size=(5,5,5), stride=2),
                nn.Conv3d(in_channels=32, out_channels=32, kernel_size=(3,3,3), stride=1),
                nn.AvgPool3d(kernel_size=(2,2,2), stride=2),
                nn.Flatten(start_dim=1),
                nn.Linear(6912, 128),
                nn.Linear(128, 25),
                nn.Linear(25,3),
                nn.ReLU()
        )
        
        self.VoxNetDecoder = nn.Sequential(
                nn.Linear(3,25),
                nn.Linear(25, 128),
                nn.Linear(128, 1000),
                nn.Unflatten(dim=1, unflattened_size=(10,10,10)),

                nn.ConvTranspose3d(in_channels=1, out_channels=32, kernel_size=(7,7,7), dilation=2),
                nn.ReLU(),
                nn.ConvTranspose3d(in_channels=32, out_channels=32, kernel_size=(5,5,5), dilation=2),
                nn.ReLU(),
                nn.ConvTranspose3d(in_channels=32, out_channels=1, kernel_size=(5,5,5), dilation=1, 
                                   output_padding=0,
                                   padding=1,
                                   stride=1),     
                nn.Tanh()
        )
        

                
    def forward(self, x):
        x = self.VoxNetEncoder(x)
        return self.VoxNetDecoder(x)
        

In [12]:
# VoxNetDecoder(VoxNetEncoder(voxel.float())).shape

In [13]:
model = VoxNetAutoencoder()


In [14]:
model(voxel.float()).shape

torch.Size([1, 32, 32, 32])

In [15]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.003)

In [16]:
loss = nn.MSELoss()

In [17]:
print("Start training ...")
model.train()

for epoch in range(30):
    overall_loss = 0
    for batch_idx, sample in enumerate(train_dataloader):
        x,y =  sample['voxel'], sample['cls_idx']
        
        optimizer.zero_grad()
        y_hat = model(x.float()).float()
        ll = loss(y_hat, y.float())
        overall_loss += ll.item()
        ll.backward()
        optimizer.step()
        
#     print("\tEpoch", epoch + 1, "complete!", "\tAverage Loss: ", overall_loss / (batch_idx*batch_size))
    print(overall_loss)
    
print("Finish!!")

Start training ...


/home/sbn/miniconda3/envs/deeplearning0/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 32, 32, 32])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 

# Test Sample:

In [18]:
model.eval()

print('Test Sample:')
for sample in train_dataloader:
    sample_x, sample_y = sample['voxel'], sample['cls_idx']
#     sample_y_hat = torch.argmax(
    bottle_neck = model.VoxNetEncoder(sample_x.float())
    print(bottle_neck)
    print('Ground Truth  =', sample_y)
#     print('Predicted     =', sample_y_hat)
#     print(voxel.shape)
    break

Test Sample:
tensor([[0.0335, 0.0000, 0.0979]], grad_fn=<ReluBackward0>)
Ground Truth  = tensor([2])


In [112]:
# Saving VoxNet model and its parameters.
torch.save(VoxNet.state_dict(), "model/AutoEncoderVoxNetmodel.params")
torch.save(VoxNet.state_dict(), "model/AutoEncoderVoxNetmodel.params.pt")
torch.save(VoxNet, "model/enitre_AutoEncoderVoxNetmodell")

In [114]:
#  (Uncomment this if you want):
# # Whenever it's needed you can do loading the trained model with*:
# model = torch.load("model/enitre_VoxNetmodel")
# model.eval()

Sequential(
  (0): Conv3d(1, 32, kernel_size=(5, 5, 5), stride=(2, 2, 2))
  (1): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (2): AvgPool3d(kernel_size=(2, 2, 2), stride=2, padding=0)
  (3): Flatten(start_dim=1, end_dim=-1)
  (4): Linear(in_features=6912, out_features=128, bias=True)
  (5): Linear(in_features=128, out_features=10, bias=True)
  (6): ReLU()
)

In [183]:
# print('Test Sample:')
# for sample in train_dataloader:
#     sample_x, sample_y = sample['voxel'], sample['cls_idx']
#     sample_y_hat = torch.argmax(model(sample_x.float()))
#     print('Ground Truth  =', sample_y)
#     print('Predicted     =', sample_y_hat)
# #     print(voxel.shape)
#     break